#### 입력 : ./make_file/division/first_round/서울시_자동차도로_분할_{숫자}.csv 
#### 출력 : ./make_file/division/second_round/서울시_도로_{숫자}.csv

## 서울시 각 동별로 도로 수가 몇 개 인지 파악하기 위한 것

In [160]:
import pandas as pd 
import requests
from tqdm import tqdm 
import os

In [161]:
# 파일이 저장된 폴더 경로
folder_path = "./make_file/division/first_round/" 
file_encoding = "UTF-8"

output_file_url = "./make_file/division/second_round/서울시_도로"

In [162]:
# 조건을 만족하는 파일 목록
valid_files = [] 

In [163]:
for temp_file_name in os.listdir(folder_path):
    file_name = temp_file_name.replace("_", " ") 
    if "서울시 자동차도로 분할" in file_name and file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, temp_file_name)  # 안전한 경로 조합

        try:
            # CSV 파일 전체 읽기
            df = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

            # 컬럼명 앞뒤 공백 제거
            df.columns = df.columns.str.strip()

            # 필요한 컬럼이 존재하는지 확인
            if {"중앙점_위도", "중앙점_경도"}.issubset(df.columns):
                check = file_name.replace("서울시 자동차도로 분할 " , "") 
                temp_check_file_url = output_file_url + "_" + check
                
                output_check = os.path.isfile(temp_check_file_url) 
                
                if output_check is False :
                    valid_files.append(file_path)
                else :
                    print(f"{temp_check_file_url} 파일이 이미 있습니다.")
        except Exception as e:
            print(f"파일 {file_name} 처리 중 오류 발생: {e}")


./make_file/division/second_round/서울시_도로_03.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_02.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_16.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_14.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_15.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_01.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_05.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_11.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_04.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_12.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_06.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_07.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_13.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_09.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_08.csv 파일이 이미 있습니다.


In [164]:
len(valid_files)

1

### kakao

In [165]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
print(open_api_key_file['사이트'].value_counts())

get_api_key = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST'].values[0][1]  

# KAKAO API KEY 값 
KAKAO_API_KEY = get_api_key  

사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
행정안전부                 1
Name: count, dtype: int64


In [166]:
def kakao_map_api(lat, lon):
    latitude = lat
    longitude = lon
    
    # 카카오 맵 API URL
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    
    # API 요청 헤더
    headers = {
        "Authorization": f"KakaoAK {KAKAO_API_KEY}"
    }
    
    # API 호출
    response = requests.get(url, headers=headers)
    data = response.json()

    # 응답이 유효한지 확인
    if 'documents' in data and len(data['documents']) > 0:
        region_3depth_name = data['documents'][0]['address']['region_3depth_name'] 
        
    else:
        # 데이터가 없을 경우 기본값 처리 
        return None
    
    return region_3depth_name

In [ ]:
for file_path in valid_files:  
    data = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

    lat_list = data["중앙점_위도"].to_list()
    lon_list = data["중앙점_경도"].to_list() 

    is_none = 0
    
    dong_list = [] 
    print(f"작업 중 : {file_path}")
    for i in tqdm(range(len(lat_list))):
        lat = lat_list[i]
        lon = lon_list[i]
        dong = kakao_map_api(lat, lon)
        dong_list.append(dong)   
        if dong is None :
            is_none += 1
            
    print(f"불러온 행정동의 수 : {len(dong_list)}")
    print(f"None의 수 : {is_none}")

    data["행정동"] = dong_list
    
    try :
        temp_out_url = file_path.replace("./make_file/division/first_round/서울시_자동차도로_분할_" , "") 
        save_file_url = output_file_url + "_" + temp_out_url 
        data.to_csv(save_file_url, index=False)
        print(f"{save_file_url} 파일 저장 완료.", end ="\n\n")
    except OSError as e :
        print(e)

작업 중 : ./make_file/division/first_round/서울시_자동차도로_분할_10.csv


  0%|▏                                       | 38/10000 [00:03<14:56, 11.11it/s]